In [ ]:
!pip install kaggle


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D


In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"parambarodia","key":"cce036f5baee749e420652a1a4e1807f"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets list


ref                                                                   title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
arnabchaki/data-science-salaries-2023                                 Data Science Salaries 2023 💸                         25KB  2023-04-13 09:55:16          28800        807  1.0              
tawfikelmetwally/automobile-dataset                                   Car information dataset                               6KB  2023-05-28 18:26:48           2511         70  0.9411765        
fatihb/coffee-quality-data-cqi                                        Coffee Quality Data (CQI May-2023)                   22KB  2023-05-12 13:06:39           4921        100  1.0              
mohithsairamreddy/salary-data 

In [ ]:
!kaggle datasets download -d nasrulhakim86/coughvid-wav
!unzip coughvid-wav.zip


Streaming output truncated to the last 5000 lines.
  inflating: public_dataset/e8e7a8fc-a199-459b-8b47-96aae07cafdc.wav  
  inflating: public_dataset/e8eaf768-9997-4a63-ad2e-4a01c5adee86.json  
  inflating: public_dataset/e8eaf768-9997-4a63-ad2e-4a01c5adee86.wav  
  inflating: public_dataset/e8ebf8aa-107c-4f94-b24b-526e4f5f1b64.json  
  inflating: public_dataset/e8ebf8aa-107c-4f94-b24b-526e4f5f1b64.wav  
  inflating: public_dataset/e8f0479e-9e75-41e7-806d-dfad3b86ac82.json  
  inflating: public_dataset/e8f0479e-9e75-41e7-806d-dfad3b86ac82.wav  
  inflating: public_dataset/e8f243e4-d456-4b40-bf1e-b6bab899453f.json  
  inflating: public_dataset/e8f243e4-d456-4b40-bf1e-b6bab899453f.wav  
  inflating: public_dataset/e8f45f08-6e1d-4c06-ae96-ce9fe43c91d1.json  
  inflating: public_dataset/e8f45f08-6e1d-4c06-ae96-ce9fe43c91d1.wav  
  inflating: public_dataset/e8f4b378-f21c-40a6-add1-2831323f544e.json  
  inflating: public_dataset/e8f4b378-f21c-40a6-add1-2831323f544e.wav  
  inflating: public_

In [ ]:
# # Load the dataset
# df = pd.read_csv('coughvid.csv')

# # Preprocess the audio files
# def preprocess_audio(filepath):
#     audio, sr = librosa.load(filepath, sr=None)
#     mfccs = librosa.feature.mfcc(audio, sr=sr, n_mfcc=40)
#     return mfccs

# # Apply preprocessing to all audio files
# df['mfcc'] = df['file_path'].apply(preprocess_audio)
# Load the metadata
metadata = pd.read_csv('public_dataset/metadata_compiled.csv')

def preprocess_audio(filepath):
    audio, sr = librosa.load(filepath, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    return mfccs


# Apply preprocessing to all audio files
metadata['mfcc'] = metadata['uuid'].apply(lambda x: preprocess_audio(f'public_dataset/{x}.wav'))



In [ ]:
# Determine the maximum length of MFCC arrays
max_length = max(len(mfcc) for mfcc in metadata['mfcc'])

# Pad or truncate MFCC arrays to the maximum length
X = np.array([np.pad(mfcc, pad_width=((0, 0), (0, max_length - len(mfcc))), mode='constant') for mfcc in metadata['mfcc']])

# Convert labels to categorical values
le = LabelEncoder()
y = to_categorical(le.fit_transform(metadata['label']))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
